# Problem
This project is a solution to the hackathon of Kin Analytics company. The problem is to predict which customers will drop a product in less than two years based on their use of the company's products and financial data such as bank account transactions and credit bureau scores.
## Data
The data to work are four datasets with information (Data aren't in GitHub repository for privacy and size reasons.): 
1) clients: This is a historical database of clients that have purchased the product “Kin Safety” from 2012 to the delivery date.
2) clientProducts: Historical of the products the client had with the Kin Security at the moment he/she applied to the new product.
3) transactions: The transactions of clients in the pre-application moment.
4) creditScore: Bureau score of clients 6 months before and 6 months after the client applied to the new product.

In [1]:
# All packages to work in the problem
import pandas as pd
import numpy as np

In [14]:
# Read all datasets with CustomerId as the index to simplify merge
clients = pd.read_csv("C:/Users/DavidG/Documents/Datacfp/BASE DE DATOS/clients_table.txt", index_col = "CustomerId")
clientProducts = pd.read_csv("C:/Users/DavidG/Documents/Datacfp/BASE DE DATOS/products_table.txt", index_col = "CustomerId")
transactions = pd.read_csv("C:/Users/DavidG/Documents/Datacfp/BASE DE DATOS/transactions_table.txt", index_col = "CustomerId")
creditScore = pd.read_csv("C:/Users/DavidG/Documents/Datacfp/BASE DE DATOS/credit_score_table.txt", index_col = "CustomerId")

In [21]:
clients.head()

,Surname,Geography,Gender,HasCrCard,IsActiveMember,EstimatedSalary,application_date,exit_date,birth_date
CustomerId,,,,,,,,,
15745584,EIRLS,Germany,Female,0.0,1.0,0.00,2018-12-14,NaN,1997-09-18
14990118,MOLOCK,Italy,Male,1.0,0.0,121219.28,2019-01-08,NaN,1980-08-03
14733224,PAWLUCH,Italy,Female,1.0,1.0,159663.59,2012-08-01,2013-08-09,1977-08-19
14648573,NALLS,Spain,Male,1.0,0.0,140827.98,2019-06-19,NaN,1979-02-27
15365443,EBERLE,Italy,Male,1.0,0.0,35521.28,2014-01-26,2015-12-04,1972-12-21
